In [30]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from astropy.modeling import fitting, models
from scipy.integrate import quadrature,trapz
import seaborn as sb
import os
from io import BytesIO as bytesio
from ipywidgets import widgets

sb.set_style('dark')
matplotlib.rcParams['font.size']=12
matplotlib.rcParams['figure.figsize']=(10,10)

# **Exposure Time Calculator**

In [4]:
def coeff_calc(x0,xn,x=None,y=None,mode = None):
    if mode == 'Gaussian':
        model = models.Gaussian1D(mean = (x0+xn)*0.5, stddev = xn-x0)
        int_y,err = quadrature(model,x0,xn)
        return int_y/(xn-x0)
    else :
        return trapz(y,x)/(xn-x0)

In [5]:
def exposure_time(det_params,M,SNR):
    wavelength     = det_params['wavelength']
    bandwidth      = det_params['bandwidth']
    effective_area = det_params['effective_area']
    M_sky          = det_params['sky_brightness']
    plate_scale    = det_params['plate_scale']
    aperture       = det_params['aperture']
    dark_current   = det_params['dark_current']
    read_noise     = det_params['read_noise']

    F_0_p   = 1.51e3*(bandwidth/wavelength)*3631*effective_area
    F_m_p   = F_0_p*pow(10,-0.4*M)
    M_sky_p = M_sky - 2.5*np.log10(plate_scale**2)
    F_sky_p = F_0_p*pow(10,-0.4*M_sky_p)

    n_pix   = np.pi*((0.5*aperture)/plate_scale)**2

    A =  (F_m_p/SNR)**2
    B = -(F_m_p + F_sky_p*n_pix + dark_current*n_pix)
    C = -n_pix*(read_noise)**2

    t1 = (-B + np.sqrt(B**2 - 4*A*C))/(2*A)
    t2 = (-B - np.sqrt(B**2 - 4*A*C))/(2*A)
    t = np.where(t1>0,t1/60,t2/60)

    return t

In [90]:
df = pd.read_csv('data/detector_parameters.csv')

if os.path.exists('param_doc'):
    os.system('rm -r param_doc')
    os.system('mkdir param_doc')
else:
    os.system('mkdir param_doc')
   
l1 = widgets.Label("Select Telescope Filter")
display(l1)

filter_name = widgets.Dropdown(
    options=df['filter'].values,
    value='INSIST NUV',
)
display(filter_name)

l2 = widgets.HTML("<h fontsize=10>Custom Response functions</h>")
display(l2)

l2 = widgets.HTML("<h fontsize=10>Use CSV file with columns 'wav', 'response'</h>")
display(l2)

up = widgets.FileUpload(multiple=True)
display(up)

l2 = widgets.Label("Enter AB magnitude of the source")
display(l2)
M = widgets.Text()
display(M)

l3 = widgets.Label("Enter required SNR of the source ")
display(l3)
SNR = widgets.Text()
display(SNR)

l3 = widgets.Label("Diameter of the Telescope (cm): ")
display(l3)
dia = widgets.Text()
display(dia)

btn = widgets.Button(description="Submit")

display(btn)

output = widgets.Output()

@output.capture(clear_output=True,wait=True)

def submit(b):
    if len(up.value)>0:
        for file in up.value:
            df_n = pd.read_csv(bytesio(up.value[file]['content']))
            df_n.to_csv(f'param_doc/{file}')
    
    if len(M.value)<1:
        M.value = '22'
        out = widgets.Label(f"Default AB magnitude set : 22")
        display(out)
    
    if len(SNR.value)<1:
        SNR.value = '5'
        out = widgets.Label(f"Default SNR set : 5")
        display(out)
    if float(M.value)>0 and float(SNR.value)>0 :
    
        det_params = df[df['filter']==filter_name.value].copy()
        if len(up.value)>0:
            global coeff
            coeff = []
            for file in os.listdir('param_doc'):
                if file[-4:]=='.csv':
                    df_coeff = pd.read_csv(f'param_doc/{file}')
                    coeff.append(coeff_calc(df_coeff['wav'].min(),df_coeff['wav'].max(),x=df_coeff['wav'].values,y=df_coeff['response'].values))
            coeff = np.array(coeff)
            effective_area = np.pi*(float(dia.value)/2)**2*coeff.prod()
            det_params['effective_area'] = effective_area
                
        t = exposure_time(det_params,float(M.value),float(SNR.value))[0]
        out = widgets.Label(f"Required exposure time for observing source with magnitude {M.value} and SNR {SNR.value} = {np.round(t,5)} minutes")
        display(out)
        l1 = widgets.Label("Used coefficients")
        display(l1)
        display(coeff)
    else:
        out = widgets.Label("AB magnitude and SNR should be greater than 0")
        display(out)
    
btn.on_click(submit) 

output

Label(value='Select Telescope Filter')

Dropdown(index=11, options=('UVIT NUV', 'UVIT FUV', 'GALEX NUV', 'GALEX FUV', 'LSST LU', 'LSST LG', 'EUCLID V'…

HTML(value='<h fontsize=10>Custom Response functions</h>')

HTML(value="<h fontsize=10>Use CSV file with columns 'wav', 'response'</h>")

FileUpload(value={}, description='Upload', multiple=True)

...

Text(value='')

Label(value='Diameter of the Telescope (cm): ')

Text(value='')

Button(description='Submit', style=ButtonStyle())

Output()